In [1]:
import re
import json
import pandas as pd

from platform import python_version

print(python_version())

3.8.8


In [2]:
# read file
with open('sydGrid.json', 'r') as myfile:
    sydGrid=myfile.read()

# parse file
grid = json.loads(sydGrid)

In [3]:
# read file
with open('tinyTwitter.json', 'r') as myfile:
    tinyTwitter=myfile.read()

# parse file
tweet = json.loads(tinyTwitter)

Tweets with no location information can be ignored \
Tweets with null or undefined (und) for the language attribute can be ignored

In [4]:
''' 
function 用来提取四个坐标且去重
''' 

def Find_coord_list(strings):
    coord_string = ""
    for i in strings:
        coord_string += str(i)+ " " 
    coords = re.findall(r'\-?\d+\.\d+', coord_string)
    coord_list = list(set(coords))
    latitude = sorted(coord_list, key=float)[0:2]
    longtitude = sorted(coord_list, key=float)[2:4]
    coord_list_only = sorted(coord_list, key=float)
    
    # 排序
    return latitude, longtitude, coord_list_only

In [5]:
# 提取grid坐标
lat = []
long = []
for i in range(len(grid['features'])):
    latitude = Find_coord_list(grid['features'][i]['geometry']['coordinates'])[0]
    longtitude = Find_coord_list(grid['features'][i]['geometry']['coordinates'])[1]
    lat.append(latitude)
    long.append(longtitude)
print(lat)

[['-34.00412', '-33.85412'], ['-33.85412', '-33.70412'], ['-33.70412', '-33.55412'], ['-34.15412', '-34.00412'], ['-34.00412', '-33.85412'], ['-33.85412', '-33.70412'], ['-33.70412', '-33.55412'], ['-34.15412', '-34.00412'], ['-34.00412', '-33.85412'], ['-33.85412', '-33.70412'], ['-33.70412', '-33.55412'], ['-34.15412', '-34.00412'], ['-34.00412', '-33.85412'], ['-33.85412', '-33.70412'], ['-33.70412', '-33.55412'], ['-34.15412', '-34.00412']]


In [6]:
lat_only = []
for i in range(len(lat)):    
    for j in range(2):
        lat_only.append(float(lat[i][j]))
        lat_only_final = sorted(Find_coord_list(lat_only)[2], key=float, reverse = True)
        
lat_only_final = [float(i) for i in lat_only_final]

print(lat_only_final)

[-33.55412, -33.70412, -33.85412, -34.00412, -34.15412]


In [7]:
long_only = []
for i in range(len(long)):    
    for j in range(2):
        long_only.append(long[i][j])
        long_only_final = Find_coord_list(long_only)[2]

long_only_final = [float(i) for i in long_only_final]
print(long_only_final)

[150.7655, 150.9155, 151.0655, 151.2155, 151.3655]


In [8]:
# 先建好空的dict
grid_code = ['A1', 'A2', 'A3', 'A4', 
             'B1', 'B2', 'B3', 'B4',
             'C1', 'C2', 'C3', 'C4',
             'D1', 'D2', 'D3', 'D4']
props=['minlat', 'maxlat', 'minlong', 'maxlong']
grid_dic={code:{prop:[] for prop in props} for code in grid_code}
#print(grid_dic)

# 再loop填数据
c = 0
while (c < len(grid_dic)):
    for i in range(1,len(lat_only_final)):
        for j in range(1,len(long_only_final)):
            grid_dic[grid_code[c]]['minlat'] = lat_only_final[i-1]
            grid_dic[grid_code[c]]['maxlat'] = lat_only_final[i]
            grid_dic[grid_code[c]]['minlong'] = long_only_final[j-1]
            grid_dic[grid_code[c]]['maxlong'] = long_only_final[j]
            c += 1
            
grid_dic

{'A1': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'A2': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'A3': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'A4': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'B1': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'B2': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'B3': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'B4': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'C1': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'C2': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'C3': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'C4': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'D1': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'D2': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'D3': {'minlat': [], 'maxlat': [], 'minlong': [], 'maxlong': []}, 'D4': {'m

In [12]:
tweet_clean = []

tweets_dic = tweet['rows'];

for i in range(len(tweets_dic)):
    
    # 先判断有没有geo
    if tweets_dic[i]['doc']['geo'] != None:   
        
        # if 在不在范围
        if (tweets_dic[i]['doc']['geo']['coordinates'][0] >= 
            min(lat_only_final)) and (tweets_dic[i]['doc']['geo']['coordinates'][0] <= 
                                      max(lat_only_final)) and (tweets_dic[i]['doc']['geo']['coordinates'][1] 
                                                                >=  min(long_only_final)) and (tweets_dic[i]['doc']['geo']['coordinates'][1] <=  max(long_only_final)):
                
            if (tweets_dic[i]['doc']['lang'] != None) or (tweets_dic[i]['doc']['lang'] != 'und'):
            
                tweet_clean.append(tweet['rows'][i])
                
print(len(tweet_clean))     

2


In [13]:
tweet_clean

[{'id': '1212162687049883648',
  'key': ['sydney', 2020, 1, 1],
  'value': 1,
  'doc': {'_id': '1212162687049883648',
   '_rev': '1-883be9710b1baaca921608c308f986f0',
   'created_at': 'Wed Jan 01 00:04:40 +0000 2020',
   'id': 1212162687049883600,
   'id_str': '1212162687049883648',
   'text': 'Happy new year ♥️\n#cheers 🥂 #skål #2020 en Sydney Harbour Bridge https://t.co/xRwZV4kWTA',
   'truncated': False,
   'entities': {'hashtags': [{'text': 'cheers', 'indices': [18, 25]},
     {'text': 'skål', 'indices': [28, 33]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/xRwZV4kWTA',
      'expanded_url': 'https://www.instagram.com/p/B6wZoRKFgSD/?igshid=9jv06r2gw9hj',
      'display_url': 'instagram.com/p/B6wZoRKFgSD/…',
      'indices': [65, 88]}]},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://instagram.com" rel="nofollow">Instagram</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': N

## 草稿

In [ ]:
list1 = grid['features'][2]['geometry']['coordinates']

In [ ]:


f1 = ""
for i in list1[0]:
    f1 += str(i)+ " " 

coords = re.findall(r'\-?\d+\.\d+', f1)
print(coords)
coord_list = list(set(coords))
print(sorted(coord_list, key=float))

In [ ]:
tweet['rows'][227]['doc']['geo']['coordinates']

In [ ]:
if (float('-33.66') > float(coord_list[1])) & (float('-33.86') < float(coord_list[0])) :
    print('11')

In [ ]:
tweet_clean[0]['doc']['place']['bounding_box']['coordinates']

In [ ]:
tweet_clean[0]['doc']['geo']['coordinates']